<a href="https://colab.research.google.com/github/vietai/dab/blob/master/colab/Sentiment_Analysis_%2B_Back_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this colab, we walk you through how to use our code in `vietai/dab` to augment an example of a Vietnamese dataset. The dataset is relatively small: it consists of only 27,000 supervised examples in its training set - perfect for some experiments on data augmentation!

For this task, we have already trained vi --> en and en --> vi translation models and placed them on Google Cloud Storage. There is no need to train new ones unless you have more data or data that fits the downstream task  better -- more on this later.



**MIT License**

Copyright (c) [2019] [Trieu H. Trinh](https://thtrieu.github.io/)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Install dependencies.

* `gcsfuse` to mount this colab to the Google Cloud Storage. Alternatively you can also mount the colab to Google Drive if you do not have access to GCS.
* `tensor2tensor`: a library with all necessary tools to perform training/inference.

In [0]:
# Mount the bucket to colab, so that python package os can access to it.
# First we install gcsfuse to be able to mount Google Cloud Storage with Colab.
print('\nInstalling gcsfuse')
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Now we install tensor2tensor
!pip install -q -U tensor2tensor
print('All done.')

# (Maybe) Connect to 8 TPUs and mount to Cloud Storage.

You will be asked to authenticate your google account (gmail) through the browser. This is needed to have access to storages (google cloud/ google drive). Otherwise, the default Colab local storage will be used. This storage is wiped the moment you refresh the browser/ restart the runtime - which is not good for tasks that require long running and checkpointing.

In [0]:
import tensorflow as tf
import os
import re
import json
import pprint
from google_drive_downloader import GoogleDriveDownloader as gdd
import pandas as pd
from google.colab import auth

auth.authenticate_user()

# In case of TPU runtime, we need to authenticate with google cloud
if 'COLAB_TPU_ADDR' in os.environ:
  use_tpu = True
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    # Upload credentials to TPU.

    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

  print('TPU devices:')
  pprint.pprint(devices)
else:
  use_tpu, tpu_address = False, ''

Now we mount.

In [0]:
# Now we mount the local storage to the google cloud bucket.
bucket = 'vien-translation'
print('Mounting bucket {} to local.'.format(bucket))
mount_point = '/content/{}'.format(bucket)

if not os.path.exists(mount_point):
  tf.gfile.MakeDirs(mount_point)

!fusermount -u $mount_point
!gcsfuse --implicit-dirs $bucket $mount_point
!ls $mount_point

# Download Foody Sentiment Analysis dataset

Here we use data crawled from Foody.vn with 30,000 annotated examples by `streetcodevn.com`, with a half-half ratio of negative to positive examples. This dataset is used as an assignment in our VietAI Foundation class.

We split this dataset into 27,000 examples for training and the remaining 3,000 examples for testing. Use the following script to download the data. After downloading you will see `train.csv` and `test.csv`, which correspond to the train and test split.

In [0]:
foody_dir = os.path.join(mount_point, 'foody_sa')
gdd.download_file_from_google_drive(file_id='1bBcVHVPtXbzgwpkDYXC4zfTI8mek4Xsu', 
                                    dest_path='{}/assignment4-data.zip'.format(foody_dir), 
                                    unzip=True)
foody_data_dir = os.path.join(foody_dir, 'data')
!ls $foody_data_dir/t*.csv

# Examine the dataset

In [0]:
train_csv = os.path.join(foody_data_dir, 'train.csv')
train_df = pd.read_csv(train_csv)
train_df

# Extract texts

We extract the text-only part of our dataset and examine them:

In [0]:
all_texts = list(train_df['text'])

for text in all_texts[:10]:
  print(text + '\n')

This in comparison with data for our translation task (IWSLT'15) is in fact drastically different! 

To see that, let's examine the test data from IWSLT'15:

In [0]:
!head -n 32 $mount_point/raw/tst2013.vi

What you are seeing here is called "domain mismatch", where the distribution of the two datasets are (wildly) different. In the reviews, you might see:



```
* Idioms "Cá lớn nuốt cá bé" "Hãy buông đôi tay nhau ra" "Không yêu đừng nói lời cay đắng"
* New words "Gấu mình dẫn đi ăn. "
* Short-hand writing "đồ ăn ko dc ngon lm đâu"
* Teen code "HjHj ch0 n4y 4n k dc ngon l4'm :3:3"
* Capitalization "TÔI KHÔNG THÍCH ĂN QUÁN NÀY cHúT nàO!!"
* Tone marks "toi ko thich an quan nay chut nao!!"
* Grammatical error.
```

While you can hardly find such instances in the translation training data IWSLT'15. Expectedly the translation models won't be able to understand such instances and give low quality translation.

To somewhat bridge this gap in domain mismatch, let's perform some preprocessing to our Foody Sentiment Analysis dataset. This should include special character filtering, lower-casing, etc.

In [0]:
# Removes parentheses etc., and leaves only alphanumeric characters
strip_special_chars = re.compile("[^\w0-9 .,:!?]+")

def clean_sentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())


def back_translate_preprocess(text):
  # We only use the first 32 words on each line:
  lines = []
  for line in text.split('\n'):
    line = ' '.join(line.split()[:32])
    line = clean_sentences(line)
    lines.append(line)
  text = '\n'.join(lines)
  
  # Replace the _ character with a proper space.
  text = text.replace('_', ' ')
  text = text.strip()

  # Since there is a lot of upper-casing in reviews,
  # e.g. QUÁN NÀY LÀM ĂN QUÁ TỆ, ĐỒ ĂN KHÔNG SẠCH SẼ!!!
  # We might need to lower-case all the text.
  text = text.replace('Đ', 'đ')

  text = text.replace('E', 'e')
  text = text.replace('È', 'è')
  text = text.replace('É', 'é')
  text = text.replace('Ẻ', 'ẻ')
  text = text.replace('Ẽ', 'ẽ')
  text = text.replace('Ẹ', 'ẹ')

  text = text.replace('Ê', 'ê')
  text = text.replace('Ề', 'ề')
  text = text.replace('Ế', 'ế')
  text = text.replace('Ể', 'ể')
  text = text.replace('Ễ', 'ễ')
  text = text.replace('Ệ', 'ệ')

  text = text.replace('A', 'a')
  text = text.replace('À', 'à')
  text = text.replace('Á', 'á')
  text = text.replace('Ả', 'ả')
  text = text.replace('Ã', 'ã')
  text = text.replace('Ạ', 'ạ')

  text = text.replace('Ă', 'ă')
  text = text.replace('Ằ', 'ằ')
  text = text.replace('Ắ', 'ắ')
  text = text.replace('Ẳ', 'ẳ')
  text = text.replace('Ẵ', 'ẵ')
  text = text.replace('Ặ', 'ặ')

  text = text.replace('Â', 'â')
  text = text.replace('Ầ', 'ầ')
  text = text.replace('Ấ', 'ấ')
  text = text.replace('Ẫ', 'ẫ')
  text = text.replace('Ẩ', 'ẩ')
  text = text.replace('Ậ', 'ậ')

  text = text.replace('U', 'u')
  text = text.replace('Ù', 'ù')
  text = text.replace('Ú', 'ú')
  text = text.replace('Ủ', 'ủ')
  text = text.replace('Ũ', 'ũ')
  text = text.replace('Ụ', 'ụ')

  text = text.replace('Ư', 'ư')
  text = text.replace('Ừ', 'ừ')
  text = text.replace('Ứ', 'ứ')
  text = text.replace('Ử', 'ử')
  text = text.replace('Ữ', 'ữ')
  text = text.replace('Ự', 'ự')

  text = text.replace('O', 'o')
  text = text.replace('Ò', 'ò')
  text = text.replace('Ó', 'ó')
  text = text.replace('Ỏ', 'ỏ')
  text = text.replace('Õ', 'õ')
  text = text.replace('Ọ', 'ọ')
  
  text = text.replace('Ơ', 'ơ')
  text = text.replace('Ờ', 'ờ')
  text = text.replace('Ớ', 'ớ')
  text = text.replace('Ở', 'ở')
  text = text.replace('Ỡ', 'ỡ')
  text = text.replace('Ợ', 'ợ')
  
  text = text.replace('Ô', 'ô')
  text = text.replace('Ồ', 'ồ')
  text = text.replace('Ố', 'ố')
  text = text.replace('Ổ', 'ổ')
  text = text.replace('Ỗ', 'ỗ')
  text = text.replace('Ộ', 'ộ')

  # Some other simple standardization.
  text = text.replace('ko ', 'không ')
  text = text.replace('dc ', 'được ')
  text = text.lower()
  return text + '\n'
  

# Write the result to a text file:
train_text_file = os.path.join(foody_data_dir, 'foody_processed_train.txt')
review_lens = []  # number of lines in each review.
with open(train_text_file, 'w') as f:
  for text in all_texts:
    processed_text = back_translate_preprocess(text)
    f.write(processed_text + '\n')
    num_lines = processed_text.count('\n')
    review_lens.append(num_lines)
    
    
!head -n 20 $train_text_file

# Pull or Clone code from our VietAI's Github `vietai/dab`

In [0]:
src = '/content/dab'
if not os.path.exists(src):
  !git clone https://github.com/vietai/dab.git
else:
  % cd $src
  !git pull
  % cd /
  
!ls $src

# Back-translate

You can surely perform beam search decoding one sentence after another until finish. This, however, would take a very long time, even with this dataset of only 27K examples. Our code helps you with handling all possible speedups you can make:

* Decoding on ML accelerator hardware (GPU or TPU).
* Batch processing of sentences.
* Processing sentences of similar length at the same time.
* Saving intermediate results during the process to deal with Colab Notebook preemtions.

At its core, the `back_translate.py` script essentially packages two calls to `t2t-decode` from vi -> en and then en -> vi. We added the ability to save intermediate results after every single batch of data since back-translating the full dataset can take quite a long time.

In [0]:
output_file = os.path.join(foody_data_dir, 'foody_paraphrased_train.txt')

is_demo = True  #@param {type:"boolean"}
from_ckpt = '/content/vien-translation/checkpoints/translate_vien_iwslt32k_tiny/avg/'  # @param{type: "string"}
to_ckpt = '/content/vien-translation/checkpoints/translate_envi_iwslt32k_tiny/avg/'  # @param{type: "string"}


if is_demo:
  paraphrase_from_file = os.path.join(foody_data_dir, 'demo_train.txt')
  paraphrase_to_file = os.path.join(foody_data_dir, 'demo_out.txt')
  # Remove previous results to run again.
  tmp_file = paraphrase_from_file + '.tmp.en.txt'
  if tf.gfile.Exists(tmp_file):
    tf.gfile.Remove(tmp_file)
  if tf.gfile.Exists(paraphrase_to_file):
    tf.gfile.Remove(paraphrase_to_file)
else:
  paraphrase_from_file = train_text_file
  paraphrase_to_file = output_file


# Without specifying batch_size in decode_hparams, the default value will be 32.
!python $src/back_translate.py \
--model=transformer --hparams_set=transformer_tiny \
--decode_hparams="beam_size=4,alpha=0.6" \
--paraphrase_from_file=$paraphrase_from_file \
--paraphrase_to_file=$paraphrase_to_file \
--from_ckpt=$from_ckpt \
--to_ckpt=$to_ckpt \
--output_dir=/dummy/ \
--use_tpu=$use_tpu \
--cloud_tpu_name=$tpu_address

# Print some paraphrases

In [0]:
!head -n 10 $train_text_file
print('==' * 10)
!head -n 10 $output_file

# Convert back-translated text to CSV format.

First, we have to process `output_file` back into a list of reviews. Since each review can have multiple lines of text, we make use of `review_lens` recorded previously to group the lines in `output_file` back into individual reviews.

In [0]:
# First we read the text in output_file
with open(output_file, 'r') as f:
  output_lines = f.readlines()

# Now we group lines into individual reviews according to review_lens.
bt_reviews = []  # bt == 'back translated'
for review_len in review_lens:
  review = output_lines[:review_len]
  output_lines = output_lines[review_len+1:]
  # combine the lines.
  review = ''.join(review)
  bt_reviews.append(review)

Next, we put the list of reviews `bt_reviews` back into Pandas data frame and save it back to csv format.

In [0]:
bt_train_df = train_df.copy()   # bt == back translated
bt_train_df['text'] = bt_reviews

bt_train_csv = os.path.join(foody_data_dir, 'bt_train.csv')
bt_train_df.to_csv(bt_train_csv, index=False)

Done! Let's load the back-translated dataset and check to see if everything looks okay:

In [0]:
bt_train_df = pd.read_csv(bt_train_csv)
bt_train_df

# Test Time Augmentation (TTA).

If we have access to the back-translation models (vi->en & en->vi), can we make use of them during test time?

The answer is yes, one idea called "Test Time Augmentation" asks the model to average its predictions on real and synthetic examples, generated from the real ones. This technique is employed widely in competitions and reliably gives improvement on Computer Vision benchmarks.

To use TTA, here we simply repeat the above steps on the test data. Note that the release test data here does not have the `class` column so we are not cheating for using TTA. We have generated the back-translated test data in a separate colab using the same procedure presented so far. The result can be found in the same Cloud Storage:

In [0]:
bt_test_csv = os.path.join(foody_data_dir, 'bt_test.csv')
bt_test_df = pd.read_csv(bt_test_csv)
bt_test_df

# Now let's train a model on the augmented data.

Using Machine Learning to solve this dataset is a standard assignment in our VietAI's foundation class. We use methods/solution/code from this assignment, prepared by VietAI's teaching staff and is available to any student of the class. 

At the moment this material is not readily viewable to the public, but we are working towards that! For now we report the results obtained from training the solution model on the original dataset and the augmented dataset (column "with BT") here. We did not make any extra modification from the original solution and still got improvement right away! Here we report the results for two settings: (1) limited data where only 10K training examples is available, and (2) when there is a full access to 27K training examples.


|     | Normal  | with BT| with TTA |
|-----|---------|-----------------------|------------------------|
| 27k | 85.86% | 86.05%               | __86.95%__                |
| 10k | 83.48% | __85.91%__               | 85.05%                |

# Acknowledgements

This work is made possible by [VietAI](http://vietai.org/). Special thanks to [Thang Luong](http://thangluong.com), Le Cao Thang, and Hoang Quy Phat for collaborating and giving comments.

# References

1. Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems. 2017.

2. Vaswani, Ashish, et al. "Tensor2tensor for neural machine translation." arXiv preprint arXiv.

3. Sennrich, Rico, Barry Haddow, and Alexandra Birch. "Improving neural machine translation models with monolingual data." arXiv preprint arXiv:1511.06709 (2015).

4. Edunov, Sergey, et al. "Understanding back-translation at scale." arXiv preprint arXiv:1808.09381 (2018).